In [8]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim  as optim
import torch.utils.data

In [9]:
X_test = np.load("../Data/X_test.npy")
y_test = np.load("../Data/y_test.npy")
person_train_valid = np.load("../Data/person_train_valid.npy")
X_train_valid = np.load("../Data/X_train_valid.npy")
y_train_valid = np.load("../Data/y_train_valid.npy")
person_test = np.load("../Data/person_test.npy")

In [10]:
Y_train = []
for y in y_train_valid:
    value = np.abs(769-y)
    Y_train.append(value)
Y_train = np.array(Y_train)

Y_test = []
for y in y_test:
    value = np.abs(769-y)
    Y_test.append(value)
Y_test = np.array(Y_test)

In [11]:
print(y_train_valid.shape)
print(Y_train.shape)
print(Y_test.shape)
print(Y_test.shape)

(2115,)
(2115,)
(443,)
(443,)


In [60]:
# create feature and targets tensor for train set. we create tensor, then we will create variable
featuresTrain = torch.from_numpy(X_train_valid)
targetsTrain = torch.from_numpy(Y_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
featuresTest = torch.from_numpy(X_test)
targetsTest = torch.from_numpy(Y_test).type(torch.LongTensor) # data type is long

In [61]:
# batch_size, epoch and iteration
batch_size = 100
n_iters = 10000
num_epochs = n_iters / (len(X_train_valid) / batch_size)
num_epochs = int(num_epochs)

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
test = torch.utils.data.TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)


In [62]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim
        
        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, 
                          nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        return out

In [64]:
# Create RNN
dtype = torch.FloatTensor
input_dim = 1000    # input dimension
hidden_dim = 200  # hidden layer dimension
layer_dim = 6     # number of hidden layers
output_dim = 10   # output dimension

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

# Cross Entropy Loss 
error = nn.CrossEntropyLoss().type(dtype)


# SGD Optimizer
learning_rate = 0.05
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
seq_dim = 25  
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
for epoch in range(num_epochs):
    for i, (signals, labels) in enumerate(train_loader):
        train  = Variable(signals.view(-1, seq_dim, input_dim))
        labels = Variable(labels )
            
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(train.float())
        
        # Calculate softmax and ross entropy loss
        loss = error(outputs, labels)
        
        # Calculating gradients
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
        
        if count % 10 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for signals, labels in test_loader:
                signals = Variable(signals.view(-1, seq_dim, input_dim))
                
                # Forward propagation
                outputs = model(signals.float())
                
                # Get predictions from the maximum value
                predicted = torch.max(outputs.data, 1)[1]
                
                # Total number of labels
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / float(total)
            
            # store loss and iteration
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            if count % 1 == 0:
                # Print Loss
                print('Iteration: {}  Loss: {}  Accuracy: {} %'.format(count, loss.data, accuracy))
                

Iteration: 10  Loss: 1.5840939283370972  Accuracy: 30 %
Iteration: 20  Loss: 1.3395147323608398  Accuracy: 39 %
Iteration: 30  Loss: 1.3117599487304688  Accuracy: 52 %
Iteration: 40  Loss: 1.456087589263916  Accuracy: 58 %
Iteration: 50  Loss: 1.2119991779327393  Accuracy: 59 %
Iteration: 60  Loss: 0.45253363251686096  Accuracy: 61 %
Iteration: 70  Loss: 0.8311282992362976  Accuracy: 59 %
Iteration: 80  Loss: 0.40881621837615967  Accuracy: 62 %
Iteration: 90  Loss: 0.9112553596496582  Accuracy: 58 %
Iteration: 100  Loss: 0.4276775121688843  Accuracy: 58 %
Iteration: 110  Loss: 0.0912146270275116  Accuracy: 53 %
Iteration: 120  Loss: 0.5784710049629211  Accuracy: 55 %
Iteration: 130  Loss: 0.5351635813713074  Accuracy: 60 %
Iteration: 140  Loss: 0.7218586206436157  Accuracy: 57 %
Iteration: 150  Loss: 0.20522741973400116  Accuracy: 58 %
Iteration: 160  Loss: 0.3935294449329376  Accuracy: 54 %
Iteration: 170  Loss: 0.214603990316391  Accuracy: 58 %
Iteration: 180  Loss: 0.305254131555557

Iteration: 1400  Loss: 3.369331534486264e-05  Accuracy: 55 %
Iteration: 1410  Loss: 0.00014613628445658833  Accuracy: 55 %
Iteration: 1420  Loss: 0.00010948658018605784  Accuracy: 55 %
Iteration: 1430  Loss: 3.814697322468419e-07  Accuracy: 55 %
Iteration: 1440  Loss: 5.188941941014491e-05  Accuracy: 55 %
Iteration: 1450  Loss: 0.0001528930733911693  Accuracy: 55 %
Iteration: 1460  Loss: 6.967544322833419e-05  Accuracy: 55 %
Iteration: 1470  Loss: 4.413604619912803e-05  Accuracy: 55 %
Iteration: 1480  Loss: 8.425712439930066e-05  Accuracy: 55 %
Iteration: 1490  Loss: 3.2482148526469246e-05  Accuracy: 55 %
Iteration: 1500  Loss: 1.2245178368175402e-05  Accuracy: 55 %
Iteration: 1510  Loss: 1.202583280246472e-05  Accuracy: 55 %
Iteration: 1520  Loss: 5.824089021189138e-05  Accuracy: 55 %
Iteration: 1530  Loss: 4.218578396830708e-05  Accuracy: 55 %
Iteration: 1540  Loss: 1.2715658215256553e-07  Accuracy: 55 %
Iteration: 1550  Loss: 2.192497231590096e-05  Accuracy: 55 %
Iteration: 1560  Lo

Iteration: 2770  Loss: 3.643035825007246e-06  Accuracy: 56 %
Iteration: 2780  Loss: 2.9468535558407893e-06  Accuracy: 56 %
Iteration: 2790  Loss: 1.9359588350198464e-06  Accuracy: 56 %
Iteration: 2800  Loss: 3.824234227067791e-06  Accuracy: 56 %
Iteration: 2810  Loss: 1.535415663056483e-06  Accuracy: 56 %
Iteration: 2820  Loss: 7.438659395120339e-07  Accuracy: 56 %
Iteration: 2830  Loss: 6.294250738392293e-07  Accuracy: 56 %
Iteration: 2840  Loss: 3.776550329348538e-06  Accuracy: 56 %
Iteration: 2850  Loss: 2.174377414121409e-06  Accuracy: 56 %
Iteration: 2860  Loss: 0.0  Accuracy: 56 %
Iteration: 2870  Loss: 1.325607286162267e-06  Accuracy: 56 %
Iteration: 2880  Loss: 3.1375884645967744e-06  Accuracy: 56 %
Iteration: 2890  Loss: 2.6226043701171875e-06  Accuracy: 56 %
Iteration: 2900  Loss: 1.6880035218491685e-06  Accuracy: 56 %
Iteration: 2910  Loss: 3.3664703096292214e-06  Accuracy: 56 %
Iteration: 2920  Loss: 1.306533818024036e-06  Accuracy: 56 %
Iteration: 2930  Loss: 6.86645535097

Iteration: 4150  Loss: 1.335144048653092e-07  Accuracy: 56 %
Iteration: 4160  Loss: 1.0490417707842425e-06  Accuracy: 56 %
Iteration: 4170  Loss: 5.435943535303522e-07  Accuracy: 56 %
Iteration: 4180  Loss: 0.0  Accuracy: 56 %
Iteration: 4190  Loss: 3.4332276754867053e-07  Accuracy: 56 %
Iteration: 4200  Loss: 7.724761985627993e-07  Accuracy: 56 %
Iteration: 4210  Loss: 7.820129326319147e-07  Accuracy: 56 %
Iteration: 4220  Loss: 4.3869019350495364e-07  Accuracy: 56 %
Iteration: 4230  Loss: 1.0013579867518274e-06  Accuracy: 56 %
Iteration: 4240  Loss: 3.7193296975601697e-07  Accuracy: 56 %
Iteration: 4250  Loss: 2.288818308215923e-07  Accuracy: 56 %
Iteration: 4260  Loss: 1.1444091541079615e-07  Accuracy: 56 %
Iteration: 4270  Loss: 9.441375823371345e-07  Accuracy: 56 %
Iteration: 4280  Loss: 5.149841513230058e-07  Accuracy: 56 %
Iteration: 4290  Loss: 0.0  Accuracy: 56 %
Iteration: 4300  Loss: 3.1471253691961465e-07  Accuracy: 56 %
Iteration: 4310  Loss: 6.961822691664565e-07  Accurac

Iteration: 5530  Loss: 2.574920756615029e-07  Accuracy: 56 %
Iteration: 5540  Loss: 1.1444091541079615e-07  Accuracy: 56 %
Iteration: 5550  Loss: 3.337860050578456e-07  Accuracy: 56 %
Iteration: 5560  Loss: 1.23977656585339e-07  Accuracy: 56 %
Iteration: 5570  Loss: 5.7220457705398076e-08  Accuracy: 56 %
Iteration: 5580  Loss: 9.53674295089968e-09  Accuracy: 56 %
Iteration: 5590  Loss: 3.2424927098873013e-07  Accuracy: 56 %
Iteration: 5600  Loss: 1.5258788721439487e-07  Accuracy: 56 %
Iteration: 5610  Loss: 0.0  Accuracy: 56 %
Iteration: 5620  Loss: 1.1444091541079615e-07  Accuracy: 56 %
Iteration: 5630  Loss: 2.1934509675247682e-07  Accuracy: 56 %
Iteration: 5640  Loss: 2.47955313170678e-07  Accuracy: 56 %
Iteration: 5650  Loss: 1.1444091541079615e-07  Accuracy: 56 %
Iteration: 5660  Loss: 3.1471253691961465e-07  Accuracy: 56 %
Iteration: 5670  Loss: 1.23977656585339e-07  Accuracy: 56 %
Iteration: 5680  Loss: 5.7220457705398076e-08  Accuracy: 56 %
Iteration: 5690  Loss: 9.536742950899

Iteration: 6930  Loss: 0.0  Accuracy: 56 %
Iteration: 6940  Loss: 3.814697180359872e-08  Accuracy: 56 %
Iteration: 6950  Loss: 7.629394360719743e-08  Accuracy: 56 %
Iteration: 6960  Loss: 8.583069188716763e-08  Accuracy: 56 %
Iteration: 6970  Loss: 1.907348590179936e-08  Accuracy: 56 %
Iteration: 6980  Loss: 1.23977656585339e-07  Accuracy: 56 %
Iteration: 6990  Loss: 1.907348590179936e-08  Accuracy: 56 %
Iteration: 7000  Loss: 1.907348590179936e-08  Accuracy: 56 %
Iteration: 7010  Loss: 0.0  Accuracy: 56 %
Iteration: 7020  Loss: 1.23977656585339e-07  Accuracy: 56 %
Iteration: 7030  Loss: 3.814697180359872e-08  Accuracy: 56 %
Iteration: 7040  Loss: 0.0  Accuracy: 56 %
Iteration: 7050  Loss: 3.814697180359872e-08  Accuracy: 56 %
Iteration: 7060  Loss: 6.67572024326546e-08  Accuracy: 56 %
Iteration: 7070  Loss: 7.629394360719743e-08  Accuracy: 56 %
Iteration: 7080  Loss: 1.907348590179936e-08  Accuracy: 56 %
Iteration: 7090  Loss: 1.23977656585339e-07  Accuracy: 56 %
Iteration: 7100  Loss

Iteration: 8410  Loss: 4.7683716530855236e-08  Accuracy: 56 %
Iteration: 8420  Loss: 0.0  Accuracy: 56 %
Iteration: 8430  Loss: 0.0  Accuracy: 56 %
Iteration: 8440  Loss: 0.0  Accuracy: 56 %
Iteration: 8450  Loss: 3.814697180359872e-08  Accuracy: 56 %
Iteration: 8460  Loss: 2.8610228852699038e-08  Accuracy: 56 %
Iteration: 8470  Loss: 0.0  Accuracy: 56 %
Iteration: 8480  Loss: 0.0  Accuracy: 56 %
Iteration: 8490  Loss: 1.907348590179936e-08  Accuracy: 56 %
Iteration: 8500  Loss: 2.8610228852699038e-08  Accuracy: 56 %
Iteration: 8510  Loss: 0.0  Accuracy: 56 %
Iteration: 8520  Loss: 4.7683716530855236e-08  Accuracy: 56 %
Iteration: 8530  Loss: 0.0  Accuracy: 56 %
Iteration: 8540  Loss: 0.0  Accuracy: 56 %
Iteration: 8550  Loss: 0.0  Accuracy: 56 %
Iteration: 8560  Loss: 2.8610228852699038e-08  Accuracy: 56 %
Iteration: 8570  Loss: 2.8610228852699038e-08  Accuracy: 56 %
Iteration: 8580  Loss: 0.0  Accuracy: 56 %
Iteration: 8590  Loss: 0.0  Accuracy: 56 %
Iteration: 8600  Loss: 1.90734859

Iteration: 10040  Loss: 9.53674295089968e-09  Accuracy: 56 %
Iteration: 10050  Loss: 0.0  Accuracy: 56 %
Iteration: 10060  Loss: 9.53674295089968e-09  Accuracy: 56 %
Iteration: 10070  Loss: 0.0  Accuracy: 56 %
Iteration: 10080  Loss: 0.0  Accuracy: 56 %
Iteration: 10090  Loss: 0.0  Accuracy: 56 %
Iteration: 10100  Loss: 2.8610228852699038e-08  Accuracy: 56 %
Iteration: 10110  Loss: 0.0  Accuracy: 56 %
Iteration: 10120  Loss: 0.0  Accuracy: 56 %
Iteration: 10130  Loss: 0.0  Accuracy: 56 %
Iteration: 10140  Loss: 9.53674295089968e-09  Accuracy: 56 %
Iteration: 10150  Loss: 9.53674295089968e-09  Accuracy: 56 %
Iteration: 10160  Loss: 0.0  Accuracy: 56 %
Iteration: 10170  Loss: 1.907348590179936e-08  Accuracy: 56 %
Iteration: 10180  Loss: 0.0  Accuracy: 56 %
Iteration: 10190  Loss: 0.0  Accuracy: 56 %
Iteration: 10200  Loss: 0.0  Accuracy: 56 %
Iteration: 10210  Loss: 2.8610228852699038e-08  Accuracy: 56 %
Iteration: 10220  Loss: 0.0  Accuracy: 56 %
Iteration: 10230  Loss: 0.0  Accuracy: 5